# ch04. word2vec 속도 개선
목표: word2vec 속도 개선

## 4.1 word2vec 개선 1️⃣
CBOW 모델: 거대한 말뭉치를 다루게 되면 문제 발생함
- 입력층의 원핫
    - 어휘 수가 많아지면 원핫 표현의 벡터 크기도 커짐
    - 해결) `Embedding` 계층 도입
- 은닉층 이후의 계산
    - 해결)  `Negative Sampling` 이라는 새로운 손실함수 도입
  

### 4.1.1 Embedding 계층
 `Embedding 계층`: 가중치 매개변수로부터 '단어 ID'에 해당하는 행(벡터)를 추출하는 계층


### 4.1.2 Embedding 계층 구현

In [1]:
import numpy as np

class Embedding:
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.idx = None

    def forward(self, idx):
        W, = self.params
        self.idx = idx
        out = W[idx]
        return out

    def backward(self, dout):
        dW, = self.grads
        dW[...] = 0

        np.add.at(dW, self.idx, dout)
        # or
        # for i, word_id in enumerate(self.idx):
        #     dW[word_id] += dout[i]

        return None


## 4.2 word2vec 개선 2️⃣
은닉층 이후의 처리..
-> 병목 어떻게 개선? **네거티브 샘플링** 을 이용해서


### 4.2.1 은닉층 이후 계산의 문제점
- 은닉층의 뉴런과 가중치 행렬(W_out)의 곱
- Softmax 계층의 계산

=> 어휘 수에 비례해 계산 증가

### 4.2.2 다중 분류에서 이진 분류로
- 네거티브 샘플링 기법의 핵심
 - **이진분류** : 다중 분류를 이진 분류로 근사하는 것이 네거티브 샘플링을 이해하는데 중요함
    - 은닉층과 출력 측의 가중치 행렬의 내적은 "say"에 해당하는 열만을 추출
    - 그 추출된 벡터와 은닉층 뉴런과의 내적을 계산하면 끝


### 4.2.3 시그모이드 함수와 교차 엔트로피 오차
이진 분류 -> 신경망
- 점수에 **시그모이드 함수**를 적용해 확률로 변환
- 손실을 구할 때에는 손실 함수로 '**교차엔트로피 오차**'를 사용
![](img/4-10.png)

    - y는 신경망이 출력한 확률
    - t: 정답 레이블
    - y-t: 두 값의 차이
    - 정답 레이블이 1이라면 y가 1에 가까워질수록 오차가 줄어든다.
    - 오차가 앞 계층으로 흘러가므로, 오차가 크면 '크게' 학습하고, 작으면 '작게' 학습하게 된다.